### Import Libraries

In [1]:
### Import Librariesfrom selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import pandas as pd

import time
from datetime import date, datetime, timedelta

import re

### Give Path and Initialize Chrome Driver

In [8]:
path = "https://qaranjobs.com/"

chrome_driver = './chromedriver.exe'

time.sleep(2)

driver = webdriver.Chrome(chrome_driver)
driver.get(path)

C:\Users\Najib\AppData\Local\Temp/ipykernel_20228/1613810267.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)


### Scroll height bar one screen height each time and sleep to wait in between

In [5]:
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

In [6]:
while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
#         time.sleep(2)
        load_more_btn = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div/div[2]/div[1]/article/div/div[2]/div/a")))
        load_more_btn.click();
# driver.quit()

ElementClickInterceptedException: Message: element click intercepted: Element <a class="load_more_jobs" href="#" style="">...</a> is not clickable at point (584, 27). Other element would receive the click: <li id="menu-item-24385" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-has-children menu-item-24385">...</li>
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008D8893+48451]
	(No symbol) [0x0086B8A1]
	(No symbol) [0x00775058]
	(No symbol) [0x007A4BA4]
	(No symbol) [0x007A36E8]
	(No symbol) [0x007A1EEB]
	(No symbol) [0x007A12FE]
	(No symbol) [0x00799AAC]
	(No symbol) [0x007BA2BC]
	(No symbol) [0x00799586]
	(No symbol) [0x007BA614]
	(No symbol) [0x007CC482]
	(No symbol) [0x007BA0B6]
	(No symbol) [0x00797E08]
	(No symbol) [0x00798F2D]
	GetHandleVerifier [0x00B38E3A+2540266]
	GetHandleVerifier [0x00B78959+2801161]
	GetHandleVerifier [0x00B7295C+2776588]
	GetHandleVerifier [0x00962280+612144]
	(No symbol) [0x00874F6C]
	(No sy

### Download whole scrolled page source locally

In [12]:
import codecs
with open('page_source.html','w',encoding='UTF-8') as f:
    f.write(driver.page_source)
    f.close()

### List of common Stem Jobs

In [24]:
stem_jobs = [
    "Software Developer",
    "Data Scientist",
    "Artificial Intelligence Engineer",
    "Machine Learning Engineer",
    "Robotics Engineer",
    "Cybersecurity Analyst",
    "Network Administrator",
    "Database Administrator",
    "Web Developer",
    "Mobile App Developer",
    "DevOps Engineer",
    "Systems Administrator",
    "Cloud Architect",
    "IT Project Manager",
    "Computer Systems Analyst",
    "Network Architect",
    "UI/UX Designer",
    "Computer Programmer",
    "Game Developer",
    "Virtual Reality Developer",
    "Augmented Reality Developer",
    "Digital Forensics Analyst",
    "Computer Hardware Engineer",
    "Embedded Systems Engineer",
    "Electronics Engineer",
    "Civil Engineer",
    "Mechanical Engineer",
    "Electrical Engineer",
    "Chemical Engineer",
    "Biomedical Engineer",
    "Aerospace Engineer",
    "Automotive Engineer",
    "Nuclear Engineer",
    "Marine Engineer",
    "Petroleum Engineer",
    "Geotechnical Engineer",
    "Structural Engineer",
    "Materials Engineer",
    "Process Engineer",
    "Industrial Engineer",
    "Operations Research Analyst",
    "Data Analyst",
    "Statistician",
    "Mathematician",
    "Actuary",
    "Cryptographer",
    "Quantitative Analyst",
    "Environmental Scientist",
    "Geoscientist",
    "Ecologist",
    "Meteorologist",
    "Oceanographer",
    "Biologist",
    "Microbiologist",
    "Biochemist",
    "Geneticist",
    "Biotechnologist",
    "Pharmaceutical Researcher",
    "Clinical Research Associate",
    "Medical Scientist",
    "Forensic Scientist",
    "Food Scientist",
    "Physicist",
    "Astrophysicist",
    "Astronomer",
    "Cosmologist",
    "Chemist",
    "Analytical Chemist",
    "Materials Scientist",
    "Science Educator",
    "STEM Consultant",
    "STEM Project Manager",
    "STEM Policy Analyst",
    "STEM Writer",
    "STEM Illustrator",
    "STEM Outreach Coordinator",
    "Senior Medical Technical Advisor",
    "Senior Health emergency preparedness and Response Advisor",
    "Health, Safety, Social and Environmental (HSSE) Officer",
    "UN Clinic Physician",
    "Project Engineer",
    "Statistics Expert",
    "CMAM Nurse"
    'IYCF Nurse',
    'ICT Support Assistant',
    'Wash Engineering',
    'WASH Project Manager',
    'IT Service Delivery Management Officer'
    'Engineer',
    'ICT Officer',
    'Information Technology Assistant (IT)',
    'ICT SPECIALIST',
    'REACH Geographic Information System (GIS) Officer',
    'Radiographer',
    'IT Coordinator',
    'Information Technology Assistant (IT)',
    'WASH Project Manager', 
    'Senior Medical Technical Advisor',
]

### Get Specific Column data for each job title

In [55]:
def extract_from_url(url, job_data):
    print(url)
    driver.get(url)
    soup=BeautifulSoup(driver.page_source, "html.parser")
    details=soup.find('div',class_='job_description').text
    match = re.search(r'(?P<educational_requirement>.*)', details)
    print(match.group('educational_requirement'))
#     regex = r"degree([\w\s,]+)\."
#     row=re.findall(regex,details)
#     if len(row)==0:
#         regex = r"master([\w\s,]+)\."
#         row=re.findall(regex,details)
#     print(row)

In [13]:
urls = []
jobs=[]
with open('page_source.html','r',encoding='UTF-8') as f:
    page_source=f.read()
    f.close()
soup = BeautifulSoup(page_source, "html.parser")
# print(soup)
job_listing=soup.find('ul',class_="job_listings")
for job in job_listing.find_all("li",class_='job_listing'):
    job_data={}
    job_data["title"]=job.find('div',class_='position').text.split('–')[0]
    if(len(job.find('div',class_='company').text))>2:
        job_data["company"]=job.find('div',class_='company').text
    else:
        job_data["company"]=job.find('div',class_='position').text.split('–')[0]
    job_data["location"]=job.find('div',class_='location').text
    job_data["posted"]=job.find('li',class_='date').text.split('on')[1]
    link = job.find('a').attrs['href']
    urls.append(link)
    job_data["url"]=link
    jobs.append(job_data)
#     extract_from_url(link,job_data)
#     print(job_data['title'])
#     time.sleep(2)
# driver.quit()

In [14]:
print(jobs)

[{'title': '\nProject Assistant ', 'company': '\nACTED ', 'location': '\n\t\t\tSomali Regional State Of Ethiopia\t\t', 'posted': ' May 21, 2023', 'url': 'https://qaranjobs.com/job/project-assistant-acted-somali-regional-state-of-ethiopia/', 'stem': 0}, {'title': '\nProtection Officer ', 'company': '\nActed ', 'location': '\n\t\t\tSomali Regional State Of Ethiopia\t\t', 'posted': ' May 21, 2023', 'url': 'https://qaranjobs.com/job/protection-officer-acted-somali-regional-state-of-ethiopia/', 'stem': 0}, {'title': '\nSenior Medical Technical Advisor ', 'company': '\nSenior Medical Technical Advisor ', 'location': '\n\t\t\tMogadishu\t\t', 'posted': ' May 21, 2023', 'url': 'https://qaranjobs.com/job/senior-medical-technical-advisor-federal-government-of-somalia-mogadishu/', 'stem': 0}, {'title': '\nSenior Health emergency preparedness and Response Advisor ', 'company': '\nSenior Health emergency preparedness and Response Advisor ', 'location': '\n\t\t\tMogadishu\t\t', 'posted': ' May 21, 20

### Format Data by labelling it as stem job or not and remove delimiters

In [95]:
import re
def split_string(x):
    return x.split('\n')[1]
def split_tab(x):
    new_string = re.sub(r'\t', '', x)
    return new_string
def categorize_stem_job(row):
    job=row['title']
    stem=0
#     print(job)
    if job.strip() in stem_jobs:
        stem=1
    else:
        stem=0
#     print(stem)
    return stem

In [96]:
cols_format=['title','company','location']
for col in cols_format:
    df[col]=df[col].apply(split_string)
df['stem']=df.apply(lambda row:categorize_stem_job(row),axis=1)
df['location']=df['location'].apply(split_tab)

In [97]:
df

,title,company,location,posted,url,stem
0,Project Assistant,ACTED,Somali Regional State Of Ethiopia,"May 21, 2023",https://qaranjobs.com/job/project-assistant-ac...,0
1,Protection Officer,Acted,Somali Regional State Of Ethiopia,"May 21, 2023",https://qaranjobs.com/job/protection-officer-a...,0
2,Senior Medical Technical Advisor,Senior Medical Technical Advisor,Mogadishu,"May 21, 2023",https://qaranjobs.com/job/senior-medical-techn...,1
3,Senior Health emergency preparedness and Respo...,Senior Health emergency preparedness and Respo...,Mogadishu,"May 21, 2023",https://qaranjobs.com/job/senior-health-emerge...,1
4,Program Assistant,Program Assistant,Bosaso,"May 21, 2023",https://qaranjobs.com/job/program-assistant-nu...,0
...,...,...,...,...,...,...
3715,District Capacity Building Consultant,Galmudug State of Somalia,"Abudwak – Galmudug , Somalia","April 7, 2022",https://qaranjobs.com/job/district-capacity-bu...,0
3716,District Capacity Building Consultant,Galmudug State of Somalia,"Balanballe – Galmudug, Somalia","April 7, 2022",https://qaranjobs.com/job/district-capacity-bu...,0
3717,"Planning Consultant Dhusamareb, Galmudug, Somalia",Galmudug State of Somalia,"Dhusamareb, Galmudug, Somalia","April 7, 2022",https://qaranjobs.com/job/planning-consultant-...,0
3718,Young graduate,Galmudug State of Somalia,"Abudwak – Galmudug, Somalia","April 7, 2022",https://qaranjobs.com/job/young-graduate-plann...,0


### Save Cleaned Data to CSV

In [98]:
df.to_csv('Qaran_jobs_raw.csv', index=False, header=True)